In [1]:
import os
import pandas as pd
import swat
import sys

In [2]:
cashost='sepviya35.aws.sas.com'
casport=5570
sess = swat.CAS(cashost, casport)

In [3]:
sess.loadactionset('datastep')
sess.loadactionset('transpose')

NOTE: Added action set 'datastep'.
NOTE: Added action set 'transpose'.


[actionset]

 'transpose'

+ Elapsed: 0.00426s, user: 0.000693s, sys: 0.000268s, mem: 0.222mb

## Transpose automatically at scale all interesting measure from line to column

In [11]:
results=sess.transpose.transpose(                                             
   table={"name":"RENAULT_ALL","caslib":"PUBLIC","groupBy":[
       {"name":"vehicle_id"},{"name":"machine"},{"name":"capteur"},{"name":"step"},{"name":"step_nb"},{"name":"sourcetimestamp"}]},
   transpose={"datavalue_NUM"},
   let="True",
   id={"name":"step_task"},
   casOut={"name":"RENAULT_ABT", "replace":True})

NOTE: There were 18166760 rows read from the table RENAULT_ALL.
NOTE: An additional 3272023 warnings were suppressed.
NOTE: The table RENAULT_ABT has 13242103 rows and 13 columns.


## Promote resulting in-memory table as public to share it for reporting and analysis and serialize it to disk for resilience

In [5]:
#sess.droptable(name="RENAULT_ABT", caslib="PUBLIC", quiet="TRUE")
#sess.promote(name="RENAULT_ABT", targetCaslib="PUBLIC")
#sess.table.save(table=dict(name='RENAULT_ABT', caslib='PUBLIC'), caslib='PUBLIC', name='RENAULT_ABT.sashdat', replace=True)

NOTE: Cloud Analytic Services saved the file RENAULT_ABT.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'RENAULT_ABT.sashdat'

+ Elapsed: 8.76s, user: 0.482s, sys: 1.65s, mem: 17.2mb

## Impute not changing sensor measure with last known in the transposed ABT

In [12]:
dscode='''
DATA casuser.RENAULT_ABT_V2(drop=cur_ActDifferentialPress cur_ActLoadingPressure cur_ActMass cur_ActResidualPressure cur_ActVacuum cur_ActVolume);
        SET casuser.RENAULT_ABT;
        by machine capteur step sourcetimestamp;
        retain cur_ActDifferentialPress cur_ActLoadingPressure cur_ActMass cur_ActResidualPressure cur_ActVacuum cur_ActVolume;

        if missing(ActDifferentialPress) then ActDifferentialPress = cur_ActDifferentialPress;
        else cur_ActDifferentialPress = ActDifferentialPress;

        if missing(ActLoadingPressure) then ActLoadingPressure = cur_ActLoadingPressure;
        else cur_ActLoadingPressure = ActLoadingPressure;

        if missing(ActMass) then ActMass = cur_ActMass;
        else cur_ActMass = ActMass;

        if missing(ActResidualPressure) then ActResidualPressure = cur_ActResidualPressure;
        else cur_ActResidualPressure = ActResidualPressure;

        if missing(ActVacuum) then ActVacuum = cur_ActVacuum;
        else cur_ActVacuum = ActVacuum;

        if missing(ActVolume) then ActVolume = cur_ActVolume;
        else cur_ActVolume = ActVolume;
RUN;''';

sess.dataStep.runCode(dscode)

[InputCasTables]

                 casLib         Name      Rows  Columns  \
 0  CASUSER(viyademo01)  RENAULT_ABT  13242103       13   
 
                                             casTable  
 0  CASTable('RENAULT_ABT', caslib='CASUSER(viyade...  

[OutputCasTables]

                 casLib            Name      Rows  Columns  Append  \
 0  CASUSER(viyademo01)  RENAULT_ABT_V2  13242103       14     NaN   
 
                                             casTable  
 0  CASTable('RENAULT_ABT_V2', caslib='CASUSER(viy...  

+ Elapsed: 206s, user: 224s, sys: 6.07s, mem: 7.52mb

## Promote resulting in-memory table as public to share it for reporting and analysis and serialize it to disk for resilience

In [13]:
sess.droptable(name="RENAULT_ABT_V2", caslib="PUBLIC", quiet="TRUE")
sess.promote(name="RENAULT_ABT_V2", targetCaslib="PUBLIC")
sess.table.save(table=dict(name='RENAULT_ABT_V2', caslib='PUBLIC'), caslib='PUBLIC', name='RENAULT_ABT_V2.sashdat', replace=True)

NOTE: Cloud Analytic Services saved the file RENAULT_ABT_V2.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'RENAULT_ABT_V2.sashdat'

+ Elapsed: 16s, user: 0.771s, sys: 3.44s, mem: 19.7mb

In [14]:
sess.terminate()